In [1]:
# ! pip install sentence-transformers

In [2]:
import torch
from torch import nn
from datasets import load_dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau
# from sklearnex import patch_sklearn
# patch_sklearn()
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

import gc
import json
import pickle
import numpy as np    
import pandas as pd
# from tqdm.notebook import tqdm
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# /kaggle/input/emotion-dataset/train_file.json

In [4]:
# PATH="/kaggle/input/"
# OUTPATH='/kaggle/working/'
# PATH="D:\\ghd\\NLP-Assignments\\Assignment4\\data\\"
PATH="data/"
OUTPATH='output/'
BATCH_SIZE=512
MAX_LENGTH=256
MAX_UTTERANCES=24
ROBERTA_LABELS=100
EPOCHS=0
EOS='</s>'
SEP='[SEP]'
# SOS='o'

torch.manual_seed(0)

model_id = "roberta-base"

In [5]:
device = ""
if torch.cuda.is_available():
    device = "cuda"
    print(torch.cuda.get_device_name(0))
else:
    device = "cpu"

device = torch.device(device)
device

NVIDIA GeForce RTX 3060 Laptop GPU


device(type='cuda')

In [6]:
pad=-100
def get_train(data):
    x=[]
    y=[]
    for row in data:
        speakers=row['speakers']
        emotions=row['emotions']
        utterances=row['utterances']
        triggers=row['triggers']
        # print(speakers,emotions,utterances)
        # trim in front
        while len(utterances)>MAX_UTTERANCES:
            utterances.pop(0)
            speakers.pop(0)
            emotions.pop(0)
            triggers.pop(0)
        
        # print(speakers,emotions,utterances)
        # pad
        while len(utterances)<MAX_UTTERANCES:
            utterances.insert(0,EOS)
            speakers.insert(0,EOS)
            emotions.insert(0,EOS)
            triggers.insert(0,-100)
        # print(speakers[1],emotions[1],utterances[1])
        t=""
        s1=[]
        s2=[]        
        for i in range(MAX_UTTERANCES):
            if(utterances[i]==EOS):
                s1.append(utterances[i])
                s2.append(utterances[i])
                t=EOS
                continue
            if(t==EOS):
                t=""
            # text.append([t,f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}"])
            s1.append(t)
            s2.append(f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}")

            t+=f"{speakers[i]}:{utterances[i]}:{emotions[i]} "
        x.append([s1,s2])
        trigs=[]
        for i in triggers:
            if(i is None):
                trigs.append(0)
            else:
                trigs.append(int(i))
        y.append(trigs)
    
    return x,y

def get_eval(data):
    x=[]
    y=[]
    max_len=max([len(i["triggers"]) for i in data])
    # left pad triggers

    for row in data:
        speakers=row['speakers']
        emotions=row['emotions']
        utterances=row['utterances']
        triggers=row['triggers']
        # trim in front
        while len(triggers)>max_len:
            triggers.pop(0)
        # left pad triggers
        while len(triggers)<max_len:
            triggers.insert(0,-100)


        # print(speakers,emotions,utterances)
        # trim in front
        while len(utterances)>MAX_UTTERANCES:
            utterances.pop(0)
            speakers.pop(0)
            emotions.pop(0)
        
        # print(speakers,emotions,utterances)
        # pad
        while len(utterances)<MAX_UTTERANCES:
            utterances.insert(0,EOS)
            speakers.insert(0,EOS)
            emotions.insert(0,EOS)
        # print(speakers[1],emotions[1],utterances[1])
        s1=[]
        s2=[]
        t=""
        for i in range(MAX_UTTERANCES):
            if(utterances[i]==EOS):
                s1.append(utterances[i])
                s2.append(utterances[i])
                t=EOS
                continue
            if(t==EOS):
                t=""
            # text.append([t,f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}"])
            s1.append(t)
            s2.append(f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}")
            t+=f"{speakers[i]}:{utterances[i]}:{emotions[i]} "
        x.append([s1,s2])
        trigs=[]
        for i in triggers:
            if(i is None):
                trigs.append(0)
            else:
                trigs.append(int(i))
        y.append(trigs)
    
    return x,y


def load_data(PATH):
    with open(PATH) as f:
        data = json.load(f)
        return data
    
label_encoding = {
    "S":0,
    "surprise":1,
    "fear":2,
    "neutral":3,
    "sadness":4,
    "disgust":5,
    "anger":6,
    "joy":7
}


In [7]:

data = load_data(PATH+"train_file.json")
data_val = load_data(PATH+"val_file.json")
x_train,y_train = get_train(data)
x_val,y_val = get_train(data_val)

In [8]:
x_train[0],y_train[0]

([['</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '',
   'Phoebe:You-you\x85you had sex with Ursula?!:surprise ',
   'Phoebe:You-you\x85you had sex with Ursula?!:surprise Eric:Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and:fear ',
   "Phoebe:You-you\x85you had sex with Ursula?!:surprise Eric:Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and:fear Phoebe:You didn't notice she was wearing different clothes?!:surprise ",
   "Phoebe:You-you\x85you had sex with Ursula?!:surprise Eric:Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and:fear Phoebe:You didn't notice she was wearing different clothes?!:surprise Eric:Well I was just so excited to see you.:sadness "],
  ['</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '<

In [9]:
max_len=0
for i in range(len(x_train)):
    for j in range(len(x_train[i][0])):
        max_len=max(max_len,len(x_train[i][0][j].split())+len(x_train[i][1][j].split()))
print(max_len)


263


In [10]:
x_val[0],y_val[0]

([['</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '</s>',
   '',
   'Chandler:Hey! Hold on a minute, hold on a second. Do you think these pearls are nice?:anger ',
   "Chandler:Hey! Hold on a minute, hold on a second. Do you think these pearls are nice?:anger Joey:I'd really prefer a mountain bike.:neutral ",
   "Chandler:Hey! Hold on a minute, hold on a second. Do you think these pearls are nice?:anger Joey:I'd really prefer a mountain bike.:neutral Chandler:Janice's birthday is coming up, I want to get her something speacial. Come in here with me.:neutral ",
   "Chandler:Hey! Hold on a minute, hold on a second. Do you think these pearls are nice?:anger Joey:I'd really prefer a mountain bike.:neutral Chandler:Janice's birthday is coming up, I want to get her something speacial. Come in here with me.:neutral Joey:Whoa, whoa, whoa, wait, whoa.:surprise ",
   "Chandler:Hey! Hold on a minute, hold on a second. Do you think these pearls are nice?:anger Joey:I'd really pr

In [11]:
# np.array(x_train).shape

In [12]:
len(x_train),len(y_train),len(x_val),len(y_val)

(6740, 6740, 843, 843)

In [13]:
tokenizer = RobertaTokenizer.from_pretrained(model_id)

In [14]:
# class

In [15]:
# len(x_train)
# len(x_train[0])
# x_train = np.array(x_train)
# x_train
y_train[0],y_train[4]

([-100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  1,
  1,
  0,
  0,
  0],
 [-100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  0,
  0,
  1])

In [16]:
class Dataset(Dataset):
    def __init__(self, data, tokenizer,labels):
        self.data = data
        self.tokenizer = tokenizer
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
#         print(self.data[idx])
#         dat = "".join(self.data[idx])
        s1,s2 = self.data[idx][0],self.data[idx][1]
        s1_tokenized = self.tokenizer(s1,max_length=MAX_LENGTH,truncation=True,return_tensors="pt",padding="max_length")
        s2_tokenized = self.tokenizer(s2,max_length=MAX_LENGTH,truncation=True,return_tensors="pt",padding="max_length")
        input_ids = torch.cat((s1_tokenized["input_ids"],s2_tokenized["input_ids"]),dim=1).squeeze(0)
        # x_tokenized = self.tokenizer(self.data[idx],max_length=MAX_LENGTH,truncation=True,return_tensors="pt",padding="max_length")
        # input_ids = x_tokenized["input_ids"].squeeze(0)
        # attention_mask = x_tokenized["attention_mask"].squeeze(0)
        return input_ids,torch.tensor(self.labels[idx]).squeeze(0)

In [17]:
train_dataset = Dataset(x_train,tokenizer,y_train)
val_dataset = Dataset(x_val,tokenizer,y_val)

In [18]:
train_dataset[0][0].shape

torch.Size([24, 512])

In [19]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=False)
val_dataloader=DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=False)

In [20]:
# class MyModel(nn.Module):
#     def __init__(self, input_size=MAX_LENGTH*2, hidden_size=MAX_UTTERANCES, num_linear_layers=MAX_UTTERANCES, linear_size=2):
#         super(MyModel, self).__init__()
        

#         self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True, num_layers=2)
        
#         self.linear_layers1 = nn.ModuleList([nn.Linear(hidden_size*2, 128) for _ in range(num_linear_layers)])
#         self.relu = nn.ReLU()
#         self.linear_layers2 = nn.ModuleList([nn.Linear(128, linear_size) for _ in range(num_linear_layers)])
#         # self.softmax_layers = nn.ModuleList([nn.Softmax(dim=1) for _ in range(num_linear_layers)])
#         self.sigmoid = nn.Sigmoid()
#     def forward(self, x):
 
#         for name, param in self.named_parameters():
#             if torch.isnan(param).any():
#                 print(name, "is nan")
#                 os.exit(0)

#         lstm_out, _ = self.lstm(x)

#         # print("Output shape of lstm = ",lstm_out.shape)

#         if(torch.isnan(lstm_out).any()):
#             print("lstm_out is nan"*10)
#             print("NAN "*10)
#             exit(0)
            
#         softmax_outputs = []
#         for i in range(len(self.linear_layers1)):
#             linear_output1 = self.linear_layers1[i](lstm_out[:, -1, :])

#             relu_output = self.relu(linear_output1)

#             linear_output2= self.linear_layers2[i](relu_output)


#             # softmax_output = self.softmax_layers[i](linear_output2)
#             sigmoid_output = self.sigmoid(linear_output2)
#             softmax_outputs.append(sigmoid_output.float())
           
#         stacked_tensor = torch.stack(softmax_outputs, dim=1)
#         return stacked_tensor
# #         return stacked_tensor.clone().detach().requires_grad_(True)

# model = MyModel()
# model.to(device)
# modnum=3
# optim = torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=0.01)
# scheduler = ReduceLROnPlateau(optim,mode='min', patience=3, factor=0.5)


In [21]:
# class MyModel(nn.Module):
#     def __init__(self, input_size=MAX_LENGTH*2, hidden_size=MAX_UTTERANCES, num_linear_layers=MAX_UTTERANCES, linear_size=2):
#         super(MyModel, self).__init__()
        

#         self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=False, num_layers=2)
        
#         self.linear_layers1 = nn.ModuleList([nn.Linear(hidden_size, 128) for _ in range(num_linear_layers)])
#         self.relu = nn.ReLU()
#         self.linear_layers2 = nn.ModuleList([nn.Linear(128, linear_size) for _ in range(num_linear_layers)])
#         # self.softmax_layers = nn.ModuleList([nn.Softmax(dim=1) for _ in range(num_linear_layers)])
#         self.sigmoid = nn.Sigmoid()
#     def forward(self, x):
 
#         for name, param in self.named_parameters():
#             if torch.isnan(param).any():
#                 print(name, "is nan")
#                 os.exit(0)

#         lstm_out, _ = self.lstm(x)

#         # print("Output shape of lstm = ",lstm_out.shape)

#         if(torch.isnan(lstm_out).any()):
#             print("lstm_out is nan"*10)
#             print("NAN "*10)
#             exit(0)
            
#         softmax_outputs = []
#         for i in range(len(self.linear_layers1)):
#             linear_output1 = self.linear_layers1[i](lstm_out[:, -1, :])

#             relu_output = self.relu(linear_output1)

#             linear_output2= self.linear_layers2[i](relu_output)


#             # softmax_output = self.softmax_layers[i](linear_output2)
#             sigmoid_output = self.sigmoid(linear_output2)
#             softmax_outputs.append(sigmoid_output.float())
           
#         stacked_tensor = torch.stack(softmax_outputs, dim=1)
#         return stacked_tensor
# #         return stacked_tensor.clone().detach().requires_grad_(True)

# model = MyModel()
# model.to(device)
# modnum=3
# optim = torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=0.01)
# scheduler = ReduceLROnPlateau(optim,mode='min', patience=3, factor=0.5)


In [22]:
# import torch
# import torch.nn as nn

# class MyModel(nn.Module):
#     def __init__(self, input_size=MAX_LENGTH*2, lstm_hidden_size=256, transformer_hidden_size=512, num_linear_layers=MAX_UTTERANCES, linear_size=2):
#         super(MyModel, self).__init__()
        
#         self.transformer_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True)
#         self.transformer_encoder = nn.TransformerEncoder(self.transformer_layer, num_layers=4)
        
#         # self.lstm = nn.LSTM(transformer_hidden_size, lstm_hidden_size, batch_first=True, bidirectional=True, num_layers=3)
        
#         self.linear_layers1 = nn.ModuleList([nn.Linear(lstm_hidden_size*2, 128) for _ in range(num_linear_layers)])
#         self.relu = nn.Relu()
#         self.linear_layers2 = nn.ModuleList([nn.Linear(128, linear_size) for _ in range(num_linear_layers)])
#         self.sigmoid = nn.Softmax(dim=1)
    
#     def forward(self, x):

#         # check if nan, 
#         # print(torch.isnan(x).any())

#         for name, param in self.named_parameters():
#             if torch.isnan(param).any():
#                 print(name, "is nan")
#                 os.exit(0)

#         transformer_out = self.transformer_encoder(x)

#         if(torch.isnan(transformer_out).any()):
#             print("transformer_out is nan"*10)
#             print("NAN "*10)
#             os.exit(0)
#             # print(transformer_out)
#         # lstm_out, _ = self.lstm(transformer_out)

#         # if(torch.isnan(lstm_out).any()):
#         #     print("lstm_out is nan"*10)
#         #     print("NAN "*10)
#         #     exit(0)
#         #     # print(lstm_out)

#         # out=lstm_out
#         out=transformer_out
#         # print("Output = ",out)
#         softmax_outputs = []
#         for i in range(len(self.linear_layers1)):
#             linear_output1 = self.linear_layers1[i](out[:, -1, :])
#             # print("linear: ",linear_output1)
#             relu_output = self.relu(linear_output1)
#             linear_output2 = self.linear_layers2[i](relu_output)
#             activation_output = self.relu(linear_output2)
#             sigmoid_output = self.sigmoid(activation_output)
#             softmax_outputs.append(sigmoid_output.float())
           
#         stacked_tensor = torch.stack(softmax_outputs, dim=1)
#         return stacked_tensor


# model = MyModel()
# model.to(device)
# modnum=4
# optim = torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=0.01)
# scheduler = ReduceLROnPlateau(optim,mode='min', patience=3, factor=0.5)


In [23]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_size=MAX_LENGTH*2, lstm_hidden_size=256, transformer_hidden_size=512, num_linear_layers=MAX_UTTERANCES, linear_size=2):
        super(MyModel, self).__init__()
        
        self.transformer_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_layer, num_layers=4)
        
        self.lstm = nn.LSTM(transformer_hidden_size, lstm_hidden_size, batch_first=True, bidirectional=True, num_layers=3)
        
        self.linear_layers1 = nn.ModuleList([nn.Linear(lstm_hidden_size*2, 128) for _ in range(num_linear_layers)])
        self.relu = nn.Tanh()
        self.linear_layers2 = nn.ModuleList([nn.Linear(128, linear_size) for _ in range(num_linear_layers)])
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):

        # check if nan, 
        # print(torch.isnan(x).any())

        for name, param in self.named_parameters():
            if torch.isnan(param).any():
                print(name, "is nan")
                os.exit(0)

        transformer_out = self.transformer_encoder(x)

        if(torch.isnan(transformer_out).any()):
            print("transformer_out is nan"*10)
            print("NAN "*10)
            os.exit(0)
            # print(transformer_out)
        lstm_out, _ = self.lstm(transformer_out)

        if(torch.isnan(lstm_out).any()):
            print("lstm_out is nan"*10)
            print("NAN "*10)
            exit(0)
            # print(lstm_out)

        # out=transformer_out
        out=lstm_out
        # print("Output = ",out)
        softmax_outputs = []
        for i in range(len(self.linear_layers1)):
            linear_output1 = self.linear_layers1[i](out[:, -1, :])
            # print("linear: ",linear_output1)
            relu_output = self.relu(linear_output1)
            linear_output2 = self.linear_layers2[i](relu_output)
            activation_output = self.relu(linear_output2)
            sigmoid_output = self.sigmoid(activation_output)
            softmax_outputs.append(sigmoid_output.float())
           
        stacked_tensor = torch.stack(softmax_outputs, dim=1)
        return stacked_tensor


model = MyModel()
model.to(device)
modnum=4
optim = torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.01)
scheduler = ReduceLROnPlateau(optim,mode='min', patience=3, factor=0.5)


In [24]:
model

MyModel(
  (transformer_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
     

In [25]:
import torch.nn as nn

criterion = nn.functional.cross_entropy

def metrics(y_true, y_pred):
    acc=accuracy_score(y_true, y_pred)
    f1_macro=f1_score(y_true, y_pred, average='macro')  
    f1_scores=f1_score(y_true, y_pred, average="weighted")
    return acc, f1_macro, f1_scores


In [26]:


def train_epoch(model, optimizer,epoch,val_dataloader,train_dataloader,metrics,weights,scheduler):
    model.train()
    losses = 0
    loss= 0
    preds = []
    actuals = []
    for batch in tqdm(train_dataloader, desc=f"Epoch:{epoch}, train_loss:{loss}",total=len(train_dataloader), leave=False):

        input_ids = batch[0].to(device)
        # attention_mask = batch[1].to(device)
        labels = batch[1].to(device)

        batch_size = input_ids.size(0)
        utt_size = input_ids.size(1)
        # print(input_ids.shape)
        outputs  = model(input_ids.float())
        # print(outputs.shape)
        outputs=outputs.view(batch_size*utt_size,-1)
        labels=labels.view(batch_size*utt_size)
        
        # print(outputs.shape,labels.shape)
        # print(outputs)

        loss = criterion(outputs, labels, weight=weights)

        loss = loss.to(device)
#         print("Loss = ",loss.item)
        losses+=loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # for name, param in model.named_parameters():
        #     if param.grad is not None:
        #         print(f'Gradient - {name}: {param.grad.norm()}')
    
        _, pred = torch.max(outputs, 1)

        # Flatten the predictions and targets
        predicted_flat = pred.view(-1)
        targets_flat = labels.view(-1)

        preds.extend(predicted_flat.cpu().numpy())
        actuals.extend(targets_flat.cpu().numpy())

        del input_ids
        # del attention_mask
        del labels
        del outputs

        gc.collect()
        torch.cuda.empty_cache()
        # break
#     print("BATCH FINISHED")
    x = losses /len(train_dataloader)
    tqdm.write(f"Epoch:{epoch}, Avg Train Loss: {x}")
    acc,macro,f1 = metrics(actuals,preds)
    tqdm.write(f"Avg Training Accuracy: {acc}, F1 Macro: {macro}, F1 Scores: {f1}")
    val_loss,acc,macro,f1,_ = evaluate(model,val_dataloader,"Validation",metrics,weights)
    scheduler.step(val_loss)
    gc.collect()
    torch.cuda.empty_cache()
    return x,val_loss




def evaluate(model,val_dataloader,name,metric,weights):

    with torch.no_grad():
        model.eval()
        losses = 0

        preds = []
        actuals = []
        for batch in tqdm(val_dataloader, desc=name,total=len(val_dataloader), leave=False):

                input_ids = batch[0].to(device)
                # attention_mask = batch[1].to(device)
                labels = batch[1].to(device)

                batch_size = input_ids.size(0)
                utt_size = labels.size(1)

                outputs  = model(input_ids.float())

                size_diff = labels.size(1) - outputs.size(1)

                # paddings=torch.zeros(batch_size,size_diff,2).to(device)


                # print(outputs.shape,paddings.shape)
                # predicted = torch.cat((outputs,paddings),dim=1)

                predicted=outputs.view(batch_size*utt_size,-1)
                labels=labels.view(batch_size*utt_size)
                
                # print(outputs.shape,labels.shape)
                # loss = criterion(predicted, labels)
                loss = criterion(predicted, labels, weight=weights)


                loss = loss.to(device)
        #         print("Loss = ",loss.item)
                losses+=loss.item()
    
                _, pred = torch.max(predicted, 1)

                # Flatten the predictions and targets
                predicted_flat = pred.view(-1)
                targets_flat = labels.view(-1)

                preds.extend(predicted_flat.cpu().numpy())
                actuals.extend(targets_flat.cpu().numpy())

                del input_ids
                # del attention_mask
                del labels
                del predicted
                # del predicted_labels
                gc.collect()
                torch.cuda.empty_cache()
            # break
        # print(losses,len(val_dataloader))
        x = losses / len(val_dataloader)
        tqdm.write(f"Avg {name} Loss: {x}")
        gc.collect()
        torch.cuda.empty_cache()
        acc,macro,f1 = metric(actuals,preds)
        tqdm.write(f"Avg {name} Accuracy: {acc}, F1 Macro: {macro}, F1 Scores: {f1}")
        return x,acc,macro,f1,preds

In [27]:
# optim = torch.optim.AdamW(model.parameters(),lr=1e-1,weight_decay=0.01)
# scheduler = None

In [28]:
a=0.5
train_loss=[]
eval_loss=[]
weights=torch.tensor([a,1-a]).to(device)
for i in range(1,EPOCHS+1):
    t,e=train_epoch(model,optim,i,val_dataloader,train_dataloader,metrics,weights,scheduler)
    train_loss.append(t)
    eval_loss.append(e)
    if(i%5==0):
        torch.save(model.state_dict(),OUTPATH+f"model_{modnum}_{i}.pth")
    # evaluate(model,val_dataloader,"Validation",metrics)

In [29]:
# # plot the training and validation loss
# import matplotlib.pyplot as plt
# plt.plot(train_loss, label='Training loss')
# plt.plot(eval_loss, label='Validation loss')
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# plt.title("Training and Validation Loss vs epoch")
# plt.legend()
# plt.show()

In [30]:
# torch.save(model, f"{OUTPATH}modelM{modnum}.pth")
# torch.save(tokenizer, f"{OUTPATH}tokenizerM{modnum}.pth")

In [31]:
# load model
loaded_model = torch.load(f"{OUTPATH}modelM{modnum}.pth")
loaded_tokenizer = torch.load(f"{OUTPATH}tokenizerM{modnum}.pth")
# weights=torch.tensor([0.1,0.9]).to(device)



In [32]:
loaded_model

MyModel(
  (transformer_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
     

In [33]:

# test
x_test, y_test = get_train(load_data(PATH+"val_file.json"))



In [34]:
test_dataset = Dataset(x_test,loaded_tokenizer,y_test)
test_dataloader=DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=False)
loss,acc,macro,f1,preds=evaluate(loaded_model,test_dataloader,"Test",metrics,weights)


Test:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:685: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  return torch._transformer_encoder_layer_fwd(


Avg Test Loss: 0.692286878824234
Avg Test Accuracy: 0.18638790035587188, F1 Macro: 0.17653674674773856, F1 Scores: 0.1303647502842283


In [35]:
y_test_list = np.array(y_test).flatten()
preds=np.array(preds).flatten()

In [36]:
dic1={}
dic2={}
for i in range(len(y_test_list)):
    if y_test_list[i] in dic1:
        dic1[y_test_list[i]]+=1
    else:
        dic1[y_test_list[i]]=1
    if preds[i] in dic2:
        dic2[preds[i]]+=1
    else:
        dic2[preds[i]]=1

dic1,dic2

({-100: 12939, 0: 6186, 1: 1107}, {0: 8487, 1: 11745})

In [37]:
p=[]
a=[]
for i in range(len(y_test_list)):
    if(y_test_list[i]!=-100):
        # print(y_test_list[i],preds[i])
        p.append(preds[i])
        a.append(y_test_list[i])

print(classification_report(a,p))
f1_s=f1_score(a,p,average=None)

for label, f1 in enumerate(f1_s):
    print(f"F1 score for label {label}: {f1}")

dic1={}
dic2={}
for i in range(len(p)):
    if p[i] in dic1:
        dic1[p[i]]+=1
    else:
        dic1[p[i]]=1
    if a[i] in dic2:
        dic2[a[i]]+=1
    else:
        dic2[a[i]]=1

dic2,dic1


              precision    recall  f1-score   support

           0       0.91      0.48      0.63      6186
           1       0.20      0.73      0.31      1107

    accuracy                           0.52      7293
   macro avg       0.55      0.60      0.47      7293
weighted avg       0.80      0.52      0.58      7293

F1 score for label 0: 0.6272226926333616
F1 score for label 1: 0.31451926819774234


({0: 6186, 1: 1107}, {1: 4031, 0: 3262})

In [38]:
print(classification_report(y_test_list, preds))
f1_scores = f1_score(y_test_list,preds, average=None)

# Print F1 score for each label
for label, f1 in enumerate(f1_scores):
    print(f"F1 score for label {label}: {f1}")

              precision    recall  f1-score   support

        -100       0.00      0.00      0.00     12939
           0       0.35      0.48      0.40      6186
           1       0.07      0.73      0.13      1107

    accuracy                           0.19     20232
   macro avg       0.14      0.40      0.18     20232
weighted avg       0.11      0.19      0.13     20232

F1 score for label 0: 0.0
F1 score for label 1: 0.40387105568050163
F1 score for label 2: 0.125739184562714


c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo